# Amazon Scraper
## ADS-509 Final
### Ivan Chavez, Uyen Pham, Lane Whitmore

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import random
import re
import itertools
from selenium.common.exceptions import NoSuchElementException


In [7]:
from selenium.webdriver.chrome.service import Service
import collections

In [2]:
# top 50 urls for movies, books and video games
m_url = 'https://www.amazon.com/Best-Sellers-Movies-TV-Prime-Video/zgbs/movies-tv/2958933011/ref=zg_bs_nav_movies-tv_1'
b_url = 'https://www.amazon.com/gp/bestsellers/books/ref=s9_acss_bw_cg_bsmpill_1c1_w?ref=bsm_nav_pill_print&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-1&pf_rd_r=56RBBA17NCWBRFTBZVMP&pf_rd_t=101&pf_rd_p=ef8cebb8-ad4b-453c-8030-a931d3822444&pf_rd_i=16857165011'
vg_url = 'https://www.amazon.com/Best-Sellers-Video-Games-Xbox-Series-X-S-Games/zgbs/videogames/20972814011/ref=zg_bs_nav_videogames_2_20972798011'

In [20]:
def scrape(url):
    # Create a WebDriver instance
    driver = webdriver.Chrome()
    driver.get(url)

    time.sleep(8)
    
    # Refresh to get past Amazon block
    driver.refresh()

    # Scroll page to find all 50
    driver.execute_script("window.scrollTo(0, 10000)") 
    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 14000)") 
    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 15000)") 
    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 16000)") 
    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 17000)") 

    # Lists to save the data to
    hrefs = []
    r_hrefs = []
    titles = []

    # Grabbing the title of each product
    title = driver.find_elements(By.XPATH,"//div[@class='_cDEzb_p13n-sc-css-line-clamp-2_EWgCb']")
    for t in title:
        titles.append(t.text)
    
    # Getting all elements that contain product href
    m_list = driver.find_elements(By.XPATH, "//a[@class='a-link-normal'][@role='link']")
    
    # All elements containing 
    mr_list = driver.find_elements(By.XPATH,"//a[@class='a-link-normal'][@title]")

    # Loop through the elements to extract the href for both 
    for i in m_list:
        href1 = i.get_attribute("href")
        hrefs.append(href1)

    for m in mr_list:
        hrefm = m.get_attribute("href")
        r_hrefs.append(hrefm)
    
    # Print lengths for feedback
    print(len(titles))
    print(len(r_hrefs))
    
    time.sleep(10)
    
    # Removing the extra hrefs
    hrefs = list(dict.fromkeys(hrefs))
    print(len(hrefs))
    
    # Close the driver
    driver.quit()

    return hrefs, r_hrefs, titles


In [21]:
m_hrefs, mr_hrefs, m_titles = scrape(m_url)

47
47
50


## *Should these be 14, 28, 36?*

In [90]:
# del m_hrefs[16]
# del m_hrefs[41]
# del m_hrefs[42]

## *Eliminate URLs without review*

In [48]:
m_asin = [url.split('/dp/')[1].split('/')[0] for url in m_hrefs]
mr_asin = [url.split('https://www.amazon.com/product-reviews/')[1].split('/')[0] for url in mr_hrefs]

#get the no review links
urls_not_in_mr = [url for url in m_hrefs if url.split('/dp/')[1].split('/')[0] in m_asin_not_in_mr]
print(urls_not_in_mr)


['https://www.amazon.com/Hypnotic-Ben-Affleck/dp/B0BX9DCNJZ/ref=zg_bs_2958933011_sccl_14/136-3510739-1229467?psc=1', 'https://www.amazon.com/Book-Club-Chapter-Diane-Keaton/dp/B0B71Y7268/ref=zg_bs_2958933011_sccl_28/136-3510739-1229467?psc=1', 'https://www.amazon.com/Hypnotic-Ben-Affleck/dp/B0BX9FHWCT/ref=zg_bs_2958933011_sccl_36/136-3510739-1229467?psc=1']


In [52]:
#filter out the no-review links
m_hrefs = [url for url in m_hrefs if url.split('/dp/')[1].split('/')[0] not in m_asin_not_in_mr]
print(len(m_hrefs))

47


## *Duplicates*

In [53]:
# Find and print the duplicates
names = [url.split('https://www.amazon.com/')[1].split('/')[0] for url in m_hrefs]
duplicates = [name for name, count in collections.Counter(names).items() if count > 1]
for duplicate in duplicates:
    print(f"Duplicate title: {duplicate}")
    indexes = [index for index, name in enumerate(names) if name == duplicate]
    for index in indexes:
        print(f"URL: {m_hrefs[index]}, Index: {index}")
    print()


Duplicate title: Super-Mario-Bros-Movie
URL: https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BXDGVMHS/ref=zg_bs_2958933011_sccl_1/136-3510739-1229467?psc=1, Index: 0
URL: https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BYL153QV/ref=zg_bs_2958933011_sccl_6/136-3510739-1229467?psc=1, Index: 5

Duplicate title: 65-Adam-Driver
URL: https://www.amazon.com/65-Adam-Driver/dp/B0B8TVD86K/ref=zg_bs_2958933011_sccl_4/136-3510739-1229467?psc=1, Index: 3
URL: https://www.amazon.com/65-Adam-Driver/dp/B0B8TVG6WW/ref=zg_bs_2958933011_sccl_42/136-3510739-1229467?psc=1, Index: 38

Duplicate title: Dungeons-Dragons-Honor-Among-Thieves
URL: https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B693QYTD/ref=zg_bs_2958933011_sccl_5/136-3510739-1229467?psc=1, Index: 4
URL: https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B6957VTZ/ref=zg_bs_2958933011_sccl_40/136-3510739-1229467?psc=1, Index: 36

Duplicate title: Spider-Man-Into-Spider-Verse-4K-UHD
URL: https://www.amazon.com/

In [67]:
m_df = pd.DataFrame({'title': m_titles, 'phref': m_hrefs, 'rhrefs': mr_hrefs})
#m_df

In [65]:
# Drop duplicates
m_df.drop_duplicates(subset="title", keep="first", inplace=True)
m_df.reset_index(drop=True, inplace=True)

In [66]:
m_df

,title,phref,rhrefs
0,The Super Mario Bros. Movie,https://www.amazon.com/Super-Mario-Bros-Movie/...,https://www.amazon.com/product-reviews/B0BXDGV...
1,John Wick: Chapter 4 - Bonus X-Ray Edition,https://www.amazon.com/John-Wick-Chapter-Bonus...,https://www.amazon.com/product-reviews/B0BX9MZ...
2,John Wick: Chapter 4,https://www.amazon.com/placeholder_title-John-...,https://www.amazon.com/product-reviews/B0B6QNS...
3,65,https://www.amazon.com/65-Adam-Driver/dp/B0B8T...,https://www.amazon.com/product-reviews/B0B8TVD...
4,Dungeons & Dragons: Honor Among Thieves,https://www.amazon.com/Dungeons-Dragons-Honor-...,https://www.amazon.com/product-reviews/B0B693Q...
5,Creed III,https://www.amazon.com/Creed-III-Michael-B-Jor...,https://www.amazon.com/product-reviews/B0B8TFG...
6,Guy Ritchie's The Covenant,https://www.amazon.com/Guy-Ritchies-Covenant-J...,https://www.amazon.com/product-reviews/B0B8P5C...
7,Spider-Man: Into the Spider-Verse,https://www.amazon.com/Spider-Man-Into-Spider-...,https://www.amazon.com/product-reviews/B07L8Q3...
8,Avatar: The Way of Water,https://www.amazon.com/Avatar-Way-Water-Sam-Wo...,https://www.amazon.com/product-reviews/B0B72TV...
9,Cocaine Bear,https://www.amazon.com/Cocaine-Bear-Keri-Russe...,https://www.amazon.com/product-reviews/B0B6JNB...


In [64]:
m_df.shape

(39, 3)

#### Very important, the products without reviews need to be manually removed here sadly. 

In [30]:
m_hrefs

['https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BXDGVMHS/ref=zg_bs_2958933011_sccl_1/136-3510739-1229467?psc=1',
 'https://www.amazon.com/John-Wick-Chapter-Bonus-X-Ray/dp/B0BX9MZFFT/ref=zg_bs_2958933011_sccl_2/136-3510739-1229467?psc=1',
 'https://www.amazon.com/placeholder_title-John-Doe/dp/B0B6QNSYRJ/ref=zg_bs_2958933011_sccl_3/136-3510739-1229467?psc=1',
 'https://www.amazon.com/65-Adam-Driver/dp/B0B8TVD86K/ref=zg_bs_2958933011_sccl_4/136-3510739-1229467?psc=1',
 'https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B693QYTD/ref=zg_bs_2958933011_sccl_5/136-3510739-1229467?psc=1',
 'https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BYL153QV/ref=zg_bs_2958933011_sccl_6/136-3510739-1229467?psc=1',
 'https://www.amazon.com/Creed-III-Michael-B-Jordan/dp/B0B8TFGZQS/ref=zg_bs_2958933011_sccl_7/136-3510739-1229467?psc=1',
 'https://www.amazon.com/Guy-Ritchies-Covenant-Jake-Gyllenhaal/dp/B0B8P5CTC3/ref=zg_bs_2958933011_sccl_8/136-3510739-1229467?psc=1',
 'https://www.amazon

In [91]:
m_df = pd.DataFrame({'title': m_titles, 'phref': m_hrefs, 'rhrefs': mr_hrefs}).drop_duplicates(subset="title", keep = False)

For some reason Amazon will list the same movie twice and list movies without reviews. This led to this manual control over the dataframe after scraping. 

In [92]:
print(len(m_df))
m_df

31


,title,phref,rhrefs
1,John Wick: Chapter 4 - Bonus X-Ray Edition,https://www.amazon.com/John-Wick-Chapter-Bonus...,https://www.amazon.com/product-reviews/B0BX9MZ...
2,John Wick: Chapter 4,https://www.amazon.com/placeholder_title-John-...,https://www.amazon.com/product-reviews/B0B6QNS...
6,Creed III,https://www.amazon.com/Creed-III-Michael-B-Jor...,https://www.amazon.com/product-reviews/B0B8TFG...
7,Guy Ritchie's The Covenant,https://www.amazon.com/Guy-Ritchies-Covenant-J...,https://www.amazon.com/product-reviews/B0B8P5C...
9,Cocaine Bear,https://www.amazon.com/Cocaine-Bear-Keri-Russe...,https://www.amazon.com/product-reviews/B0B6JNB...
10,Avatar: The Way of Water,https://www.amazon.com/Avatar-Way-Water-Sam-Wo...,https://www.amazon.com/product-reviews/B0B72TV...
12,Operation Fortune: Ruse de Guerre,https://www.amazon.com/Operation-Fortune-Guerr...,https://www.amazon.com/product-reviews/B0B7FM7...
14,Plane,https://www.amazon.com/Plane-Gerard-Butler/dp/...,https://www.amazon.com/product-reviews/B0B7C8J...
15,Jesus Revolution,https://www.amazon.com/Jesus-Revolution-Joel-C...,https://www.amazon.com/product-reviews/B0B8JB3...
16,Puss in Boots: The Last Wish,https://www.amazon.com/Puss-Boots-Last-Antonio...,https://www.amazon.com/product-reviews/B0B66B1...


## Pull book URLs

In [ ]:
b_prod, b_review = scrape(b_url)

## Pull Video Game URLs

In [ ]:
v_prod, v_review = scrape(vg_url)

## Scrape Movies' reviews and details

In [45]:
def scrape_movie_details(list_urls):

    # lists for movie details
    title = []
    imdb = []
    amazon_rating = []
    review_count = []
    length = []
    year = []
    rating = []
    price = []


    # activate driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()

        title.append(driver.find_element_by_xpath("//h1[@data-automation-id='title']").text)
        # some products that are new do not have official ratings and a bypass needs to be created. Used the NoSuchElement exception from selenium to bypass this

        # imbd rating
        try:
            im = driver.find_element_by_xpath("//span[@data-automation-id='imdb-rating-badge']").text
            imdb.append(im.split(" ")[1])
        except NoSuchElementException:
            print("No IMDB Rating")
            continue

        
        # amazon rating
        try:
            ar = driver.find_element_by_xpath("//span[@class='_3mK_sl _3NWEVD']").get_attribute("aria-label")
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        # amount of amazon reviews
        try:
            review_count.append(driver.find_element_by_xpath("//span[@class='_1Y_bt3']").text)
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        # movie runtime
        try:  
            length.append(driver.find_element_by_xpath("//span[@data-automation-id='runtime-badge']").text)
        except NoSuchElementException:
            print("No Amazon Runtime")
            length.append(np.nan)
        
        # year released
        year.append(driver.find_element_by_xpath("//span[@data-automation-id='release-year-badge']").text)

        # age rating G|PG etc..
        rating.append(driver.find_element_by_xpath("//span[@class='G8xF_x']").text)
        
        # price in whole dollar
        try:
            pr = driver.find_element_by_xpath("//button[@class='_2X_Irl tvod-button av-button']").text
            price.append(re.findall("\d+", pr)[0])
        except NoSuchElementException:
            print("No Listed Price")
            price.append(np.nan)

        time.sleep(random.randint(8,15))
    
    # close driver
    driver.close()   

    # create dataframe
    data = pd.DataFrame({"title": title, "imbd_rating": imdb,"amazon_rating":amazon_rating, "review_count": review_count, "runtime": length, 
                         "year": year, "rating": rating, "price": price})
    
    return data


        

def scrape_reviews(list_urls):

    # lists for reviews
    rating = []
    rating_fin = []
    review = [] 
    name = []
    title = []
    date_place = [] 
    movie = []

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:

        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 900):

            # using the url plus this handy snippet to loop through all review pages
            driver.get(i+f"?ie=UTF8&pageNumber={n}")
            
            time.sleep(random.randint(5,12))
            driver.refresh()

            # grabbing names past first two that are always there to decide if page has reviews
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]
            rev_check = []
            for a in names:
                rev_check.append(a.text)

            # if page has 1  or more name (reviews) scrape info below
            if len(rev_check) > 0:
                
                # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                # rating
                ratings = driver.find_elements_by_xpath("//a[@class='a-link-normal']")

                # review title
                rev_title = driver.find_elements_by_xpath("//a[@class='a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold']")

                # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                # movie name
                movies = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)

                for na in names:
                    name.append(na.text)

                for rat in ratings:
                    rating.append(rat.get_attribute("title"))

                            
                for t in rev_title:
                    title.append(t.text)
                            
                for d in dateplaces:
                    date_place.append(d.text)
                    
                for m in movies:
                    for mo in [m.text] * len(rev_check):
                        movie.append(mo)

                # give amazon a little break     
                time.sleep(random.randint(8,15))

            # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_check) == 0:
                print("There are no more reviews for this product!")
                break
    
    # cleaning up rating to remove some unwanted elements that were scraped
    for rat in rating:
        if rat != '':
            if rat.split()[1] == 'out':
                rating_fin.append(rat.split()[0])


    # close driver
    driver.close()

    # see lens to find out if there were any mistakes
    print(len(name))
    print(len(review))
    print(len(rating_fin))
    print(len(title))
    print(len(date_place))
    print(len(movie))

    # create review dataframe
    data = pd.DataFrame({"name": name, "movie_title": movie, "review_title": title, "review": review, "rating": rating_fin, "date_place": date_place})

    return data

In [93]:
movie_details_df = scrape_movie_details(m_df['phref'])

In [94]:
movie_details_df.sort_values('title', ascending=False)          

,title,imbd_rating,amazon_rating,review_count,runtime,year,rating,price
12,The Whale,7.7,4.3,"10,878",1 h 56 min,2022,18+,5
17,The Ritual Killer,4.5,3.7,"2,290",1 h 31 min,2023,16+,4
16,The Fast and the Furious,6.8,4.7,"1,214",1 h 47 min,2001,PG-13,3
26,The Croods: A New Age,6.9,4.8,"59,776",1 h 35 min,2020,PG,3
19,Shazam! Fury of the Gods,6.0,4.3,232,2 h 10 min,2023,PG-13,5
10,Scream VI,6.6,4.3,391,2 h 2 min,2023,18+,5
9,Puss in Boots: The Last Wish,7.9,4.7,"11,506",1 h 42 min,2022,PG,5
7,Plane,6.5,4.5,"9,886",1 h 47 min,2023,16+,5
14,Pitch Perfect 3,5.8,4.6,"14,958",1 h 33 min,2017,PG-13,3
6,Operation Fortune: Ruse de Guerre,6.3,4.2,886,1 h 53 min,2023,R,5


In [95]:
movie_details_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/md_df.csv')

In [96]:
movie_review_df = scrape_reviews(m_df['rhrefs'])

There are no more reviews for this product!
There are no more reviews for this product!
There are no more reviews for this product!
There are no more reviews for this product!
There are no more reviews for this product!
There are no more reviews for this product!
There are no more reviews for this product!


In [ ]:
movie_review_df.head()

In [ ]:
movie_review_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/mr_df.csv')

## Scraping Books

In [ ]:
def scrape_book_details(list_urls):
    book = []
    amazon_rating = []
    review_count = []
    length = []
    year = []
    rating = []
    price = []
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()
        book.append(driver.find_element_by_xpath("//h1[@data-automation-id='title']").text)

        # some products that are new do not have official ratings and a bypass needs to be created
        try:
            ar = driver.find_element_by_xpath("//span[@class='a-icon-alt']").text
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        try:
            review_count.append(driver.find_element_by_xpath("//span[@class='_1Y_bt3']").text)
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        try:  
            length.append(driver.find_element_by_xpath("//div[@class='a-section a-spacing-none a-text-center rpi-attribute-value']").text)
        except NoSuchElementException:
            print("No Amazon Page Length")
            length.append(np.nan)
            
        year.append(driver.find_element_by_xpath("//span[@id='productSubtitle']").text.split()[-1])

        rating.append(driver.find_element_by_xpath("//a[@class='a-link-normal'][@href='#']").text)
        
        try:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[1].text)
        except NoSuchElementException:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[0].text)

        time.sleep(random.randint(8,15))
    
    driver.close()   
    data = pd.DataFrame({"title": book,"amazon_rating":amazon_rating, "review_count": review_count, "runtime": length, 
                         "year": year, "rating": rating, "price": price})
    return data

    

## Scraping Video Games

In [ ]:
def scrape_games_details(list_urls):
    game = []
    amazon_rating = []
    review_count = []
    rating = []
    price = []
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()
        game.append(driver.find_element_by_xpath("//span[@id='productTitle']").text)

        # some products that are new do not have official ratings and a bypass needs to be created
        try:
            ar = driver.find_element_by_xpath("//span[@class='a-icon-alt']").text
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        try:
            review_count.append(driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").text.split()[0])
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        rating.append(driver.find_element_by_xpath("//a[@class='a-popover-trigger a-declarative'][@role='button']").text)

        try:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[1].text)
        except NoSuchElementException:
            price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']")[0].text)
        

        time.sleep(random.randint(8,15))
    
    driver.close()  

    data = pd.DataFrame({"title": game,"amazon_rating":amazon_rating, 
                         "review_count": review_count, "rating": rating, 
                         "price": price})
    
    return data